In [1]:
import matplotlib as mpl
# COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.dpi'] = 100


# import fancyimpute as fi
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import variation
from math import isnan

from scipy.cluster.hierarchy import dendrogram,linkage

import sys
sys.path.append("../src/")
import lumberjack
import tree_reader as tr

import sklearn

In [2]:
# counts = np.loadtxt('/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/work/johnston_log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/work/johnston_header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/aging_sc/var_filtered_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/aging_sc/var_filtered_header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/fan_tendon/log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/fan_tendon/header.txt",dtype=str)

counts = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/raw_data/vision_sc/nesterowa_counts.txt')
header = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/raw_data/vision_sc/nesterowa_gene_header.txt',dtype=str)

# counts = np.loadtxt('nesterowa_counts.txt')
# header = np.loadtxt('nesterowa_gene_header.txt',dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/nelmari/nc_filtered_log.txt')
# header = np.loadtxt('/Users/boris/taylor/nelmari/gene_header_filtered.txt',dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/testing/iris.trunc')
# header = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/testing/iris.features',dtype=str)

# feature_sort = dendrogram(linkage(counts.T,metric='correlation',method='average'),no_plot=True)['leaves']

# counts = counts[cell_sort].T[feature_sort].T
# counts = counts.T[feature_sort].T
# header = header[feature_sort]


In [ ]:

training_mask = np.random.random(counts.shape[0]) > 0
testing_mask = np.logical_not(training_mask)

training_counts = counts[training_mask]
testing_counts = counts[testing_mask]

# training_counts = counts
# testing_counts = counts

# forest = tr.Forest.reconstitute('./forest_nelmari')

forest = tr.Forest.reconstitute('./forest_vision_small')
# forest = tr.Forest.reconstitute('./forest_vision_lrg_l1')
# forest = tr.Forest.reconstitute('./forest_vision_lrg_l2')


# forest = tr.Forest.reconstitute('./forest_johnston_braid')

# forest = tr.Forest.load("../testing/nelmari/",input='../../../../../nelmari/nc_filtered_log.txt',output='../../../../../nelmari/nc_filtered_log.txt')
# forest = tr.Forest.load("../testing/nesterowa_forest/",input="../../work/nesterowa_counts.txt",output="../../work/nesterowa_counts.txt")
# forest = tr.Forest.load("../testing/johnston_forest/",input="../../work/johnston_log_counts.txt",output="../../work/johnston_log_counts.txt")

# counts = sklearn.preprocessing.scale(counts)

# forest = lumberjack.fit(training_counts,test_counts=testing_counts,trees=50,dispersion_mode="ssme",norm="l1",drop='none',sfr=1,in_feature_subsample=200,out_feature_subsample=200,sample_subsample=100,depth=5,leaves=20,header=header)
# forest = lumberjack.fit(training_counts,test_counts=testing_counts,trees=100,dispersion_mode="ssme",norm="l2",lrg_mem=True,drop='none',sfr=1,in_feature_subsample=1000,out_feature_subsample=1000,sample_subsample=200,depth=5,leaves=20,header=header)

# forest.weigh_leaves()

training_counts = forest.output
testing_counts = forest.test

# predicted = forest.predict_matrix(testing_counts)

# true_counts = testing_counts

subsampling = 1

# mask = np.loadtxt('./testing/holdout_mask_counts.txt')

# held_out_counts = np.loadtxt('./testing/held_out_counts.txt')

#2 trees at 2:17



In [ ]:
# forest.backup('./forest_johnston_ihmm')
# forest.backup('./forest_johnston_ssme')
# forest.backup('./forest_johnston_flat_sme')
# forest.backup('./forest_johnston_var')
# forest.backup('./forest_fan_ssme')

# forest.backup('./forest_johnston_braid')
# forest.backup('./forest_nelmari')

# forest.backup('./forest_vision_small')
# forest.backup('./forest_vision_l1')
# forest.backup('./forest_vision_l2')
# forest.backup('./forest_vision_lrg')



In [ ]:
forest.reset_clusters()
from scipy.cluster.hierarchy import linkage,dendrogram

feature_sort = dendrogram(linkage(counts.T,metric='correlation',method='average'),no_plot=True)['leaves']

counts = forest.output

encoding = forest.node_sample_encoding(forest.leaves())
print(encoding.shape)

print(np.sum(np.sum(encoding,axis=1) == 0))
cell_sort = dendrogram(linkage(encoding,metric='cos',method='average'),no_plot=True)['leaves']
leaf_sort = dendrogram(linkage(encoding.T,metric='cos',method='average'),no_plot=True)['leaves']
# feature_sort = dendrogram(linkage(forest.output.T,metric='cos',method='average'),no_plot=True)['leaves']
# feature_sort = np.argsort(np.var(forest.output,axis=0))


plt.figure(figsize=(10,10))
# plt.imshow(encoding,aspect='auto',cmap='binary')
plt.imshow(encoding[cell_sort].T[leaf_sort].T,aspect='auto',cmap='binary')
plt.show()


# print(np.sum(np.isnan(encoding).flatten()))

cell_clusterings = forest.cluster_samples_encoding(distance='cos',k=10,steps=50,override=False,verbose=True)
# cell_clusterings = forest.cluster_samples_simple(pca=True,subsample=1400,distance='cos',k=10,steps=50,override=True,verbose=True)
leaf_clusterings = forest.cluster_leaf_samples(distance='cos',k=10,steps=50,override=False,verbose=True)

cell_order = np.argsort(cell_clusterings)
leaf_order = np.argsort(leaf_clusterings)

# clustered_counts = forest.output[cell_order].T[feature_sort].T
clustered_counts = counts[cell_order].T[feature_sort].T
clustered_encoding = encoding[cell_order].T[leaf_order].T

plt.figure(figsize=(15,10))
plt.title("Cell x Gene Expression, Forest Clustering",fontsize=20)
plt.imshow(clustered_counts,aspect='auto')
plt.xlabel("Genes",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.colorbar()
plt.show()

plt.figure(figsize=(15,10))
plt.title("Cell x Node Organization, Forest Clustering",fontsize=20)
plt.imshow(clustered_encoding,aspect='auto',cmap='binary')
plt.xlabel("Leaves",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.show()

print("===================")

for cluster in forest.sample_clusters:
    print(len(cluster.samples))

print("===================")

for cluster in forest.leaf_clusters:
    print(len(cluster.nodes))

print("===================")
print("===================")
print("===================")
print(len(forest.sample_clusters))
print(len(forest.leaf_clusters))

In [ ]:
# forest.coordinates(override=False)
# forest.tsne_encoding(override=True)
# forest.plot_split_clusters()

# feature = "Cd34"
# coordinates = forest.coordinates(override=False,no_plot=True,pca=True)
# fi = forest.truth_dictionary.feature_dictionary[feature]
# plt.figure()
# plt.title(f"Expression of {feature}")
# plt.scatter(coordinates[:,0],coordinates[:,1],c=forest.output[:,fi],s=10)
# plt.colorbar()
# plt.show()

In [ ]:
#########################################################
#########################################################
#########################################################
##################                     ##################
##################   ##   ##   ####    ##################
##################    ## ##   ##  ##   ##################
##################     ###    ##  ##   ##################
##################     ##     ##  ##   ##################
##################    ##       ####    ##################
##################                     ##################
#########################################################
#########################################################
#########################################################


forest.reset_clusters()
forest.interpret_splits(depth=4,k=20,mode='sample',steps=50,metric='jaccard',distance='cos',optimization="max",pca=False,override=True,verbose=True)
# forest.interpret_splits(depth=5,k=10,subsample=5500,mode='gain',steps=50,distance='cos',override=False,verbose=True)
# split_order = np.argsort(forest.split_labels)
# print(np.sum(forest.split_labels < 7))
# print(np.sum(forest.split_labels < 8))
# print(np.sum(forest.split_labels < 9))

In [ ]:
# tree = forest.most_likely_tree(depth=5)
# tree

tree = forest.maximum_spanning_tree(depth=4)
tree

# transitions = forest.split_cluster_transition_matrix()
# transitions[:,88]

forest.likely_tree = forest.maximum_tree

tree


In [ ]:
forest.cluster_samples_encoding(k=20,steps=50,override=True,verbose=True)
# forest.cluster_samples_simple(pca=True,k=20,steps=50,override=True,verbose=True)
forest.plot_cell_clusters()


In [ ]:
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA

# tc = TSNE().fit_transform(forest.counts)


# tc = forest.tsne(pca=True,override=False,no_plot=True)

# tc = PCA(n_components=2).fit_transform(forest.output)

tc = forest.coordinates(no_plot=True)

for split_cluster in forest.split_clusters:
    print(split_cluster.id)
    try:
        print(sorted(list(split_cluster.braid_features().items()),key=lambda x:x[1])[::-1])
#         print(list(split_cluster.braid_features().items()))
    except:
        pass
    split_cluster.plot_cell_counts()

    sister_color = split_cluster.sister_scores()

    f = plt.figure(figsize=(15,10))
    plt.title("Sister Split")
    plt.scatter(tc[:,0],tc[:,1],c=sister_color,cmap='bwr',vmin=max(sister_color)*-1)
    plt.colorbar()
    plt.show()



In [ ]:
# forest.plot_manifold()

In [ ]:
cell_identity_str = np.loadtxt("../../raw_data/vision_sc/nesterowa_cell_type_membership.txt",dtype=str)
cell_identity_header = np.loadtxt("../../raw_data/vision_sc/nesterowa_cell_type_header.txt",dtype=str)
cell_identity_uid = cell_identity_str[:,0]
cell_identity_float = cell_identity_str.astype(dtype=float)
cell_identity_binary = cell_identity_float.astype(dtype=bool)
counts = np.loadtxt("../../raw_data/vision_sc/nesterowa_counts.txt")

In [ ]:
cell_identity_header
cell_identity_header[9:-1]
cell_identity = np.array([([0,] + list(np.arange(c.shape[0])[c] + 1))[-1] for c in cell_identity_binary[:,9:-1]])

cell_identity[:100]

In [116]:
def partition_mutual_information(p1,p2):
    p1 = p1.astype(dtype=float)
    p2 = p2.astype(dtype=float)
    population = p1.shape[1]
    intersections = np.dot(p1,p2.T)
    partition_size_products = np.outer(np.sum(p1,axis=1),np.sum(p2,axis=1))
    log_term = np.log(intersections) - np.log(partition_size_products) + np.log(population)
    log_term[np.logical_not(np.isfinite(log_term))] = 0
    mutual_information_matrix = (intersections / population) * log_term
    return mutual_information_matrix

In [117]:
from sklearn.metrics import mutual_info_score

In [118]:
# pa = [0,0,0,1,1,1,0,1]
# pb = [0,1,1,0,1,1,0,0]
pa = [0,0,0,1,1,1,0,1]
paa = [[0,0,0,1,1,1,0,1],[1,1,1,0,0,0,1,0]]
pb = [1,0,0,1,1,1,0,1]
pbb = [[1,0,0,1,1,1,0,1],[0,1,1,0,0,0,1,0]]

In [119]:
mutual_info_score(pa,pb)

0.38039566584857787

In [120]:
from scipy.stats import pearsonr
pearsonr(pa,pb)

(0.7745966692414834, 0.024008196755730942)

In [123]:
mim = partition_mutual_information(np.array(paa),np.array(pbb))

[[ 0.47000363  0.        ]
 [-0.91629073  0.69314718]]


/Users/boris/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


In [124]:
np.sum(mim)

0.38039566584857776